In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from glob import glob
import numpy as np
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

In [2]:
IMAGE_SIZE = [224,224]

In [3]:
train_path = "/content/drive/MyDrive/Computer Vision/data/train"
test_path = "/content/drive/MyDrive/Computer Vision/data/test"

In [4]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3],weights="imagenet",include_top=False)

58900480/58889256 [==============================] - 3s 0us/step


In [5]:
# dont train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [6]:
# getting number of classes
folders = glob("/content/drive/MyDrive/Computer Vision/data/train/*")
print(len(folders))

4


In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders),activation="softmax")(x)
model = Model(inputs=vgg.input,outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
from tensorflow.keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [9]:
model.compile(loss="binary_crossentropy",optimizer=sgd,metrics=["accuracy"])

In [10]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    shear_range=0.2,
    zoom_range =0.2,
    horizontal_flip = True,
    fill_mode = "nearest")

In [11]:
# Data Augmentation
test_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    shear_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest")                                      

In [12]:
train_set = train_datagen.flow_from_directory(train_path,target_size=(224,224),batch_size=32,class_mode="categorical")

Found 475 images belonging to 4 classes.


In [13]:
test_set = test_datagen.flow_from_directory(test_path,target_size=(224,224),batch_size=32,class_mode="categorical")

Found 264 images belonging to 4 classes.


In [14]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),cooldown=0,patience=5,min_lr=0.5e-6)


In [15]:
ckpt = ModelCheckpoint(filepath="vggmodel.h5",verbose =1,save_best_only= True)
callbacks = [ckpt,lr_reducer]

start = datetime.now()


In [16]:
model.fit_generator(train_set,validation_data=test_set,
                    epochs=1,
                    steps_per_epoch=5,
                    validation_steps = 32,
                    callbacks=callbacks,verbose=1)

duration = datetime.now() - start
print("Training Completed in Time: ",duration)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


5/5 [==============================] - 338s 75s/step - loss: 22.7999 - accuracy: 0.5935 - val_loss: 12.0242 - val_accuracy: 0.8220

Epoch 00001: val_loss improved from inf to 12.02415, saving model to vggmodel.h5
Training Completed in Time:  0:06:41.872601
